In [1]:
import types
import pandas as pd

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from geopy.distance import distance

print('Libraries imported.')

Libraries imported.


In [2]:
# reads file containing adresses of numerelli
indirizzi=pd.read_excel('indirizzi.xlsx')

In [3]:
geolocator = Nominatim(user_agent="to_explorer")

In [4]:
# adds latitude and longitude info to dataframe
for i in range(len(indirizzi)):
    indirizzo = indirizzi.loc[i]['Indirizzo']
    
    location = geolocator.geocode(indirizzo)
    latitude = location.latitude
    longitude = location.longitude
    
    indirizzi.loc[i, 'lat'] = latitude
    indirizzi.loc[i, 'long'] = longitude

In [5]:
indirizzi

,Nome,Indirizzo,lat,long
0,Piero,"via dei Brusati, Roma",41.880854,12.415655
1,Federico,"via Lorenzo Rocci, Roma",41.863275,12.439772
2,FabrizioC,"via Jean Paul Sartre, Roma",41.837330,12.630502
3,FabrizioG,"Largo Sri Lanka, Roma",41.825432,12.471934
4,Gigi,"via Cena, Fiumicino",41.775807,12.226396
5,Sirio,"Via Caterina Troiani, Roma",41.813189,12.452846
6,Rocco,"via Valcamonica, Roma",41.932775,12.525885
7,Sergio,"via Bepi Romagnoni, Roma",41.775748,12.343886


In [6]:
# with k-means clusters indirizzi and gets the center for the cluster
lat_long=indirizzi[['lat','long']]
id_n=1
kmeans = KMeans(n_clusters=id_n, random_state=0).fit(lat_long)
id_label_uffici=kmeans.labels_
center = np.array(kmeans.cluster_centers_)

In [7]:
# plots numerelli position and the center of the cluster
address = 'Roma'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Roma using latitude and longitude values
map_Roma = folium.Map(location=[latitude, longitude], zoom_start=11)

label='Roma'
#maxdensita=Municipi['Densita'].max()

for lat, lng, name, in zip(indirizzi['lat'], 
                                   indirizzi['long'], 
                                   indirizzi['Nome']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Roma)  

for lat, long in center:
    label = 'Baricentro Numerelli {}'.format(center)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, long],
        #icon=folium.Icon(color=colors[center]),
        #icon=folium.Icon(color='red'),
        popup=label).add_to(map_Roma)
    
    
map_Roma

In [8]:
# @hidden_cell
# You might want to remove the Foursquare credentials before you share the notebook.
CLIENT_ID = 'LULHM4VRGYYNRP0UCEDDU3M2GPA55IQ4UH5142KJIHOSKR4W' # your Foursquare ID
CLIENT_SECRET = '2ERUN2D0NN4T1PJ2VAVONFFEJWHG21IPUBM23N4XVHI0MP0I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [9]:
radius = 5000
LIMIT = 100
bari_lat=center[0][0]
bari_lng=center[0][1]

In [10]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                bari_lat, 
                bari_lng, 
                radius, 
                LIMIT)

In [11]:
results = requests.get(url).json()["response"]['groups'][0]['items']

In [12]:
venues_list=[]
venues_list.append([( 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

In [13]:
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

In [14]:
nearby_venues.columns = [
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

In [15]:
nearby_venues.groupby('Venue Category').count()['Venue'].sort_values(ascending=False).head()

Venue Category
Italian Restaurant      16
Pizza Place              9
Ice Cream Shop           8
Café                     5
Gym / Fitness Center     4
Name: Venue, dtype: int64

In [16]:
Ristoranti=nearby_venues[nearby_venues['Venue Category'].str.contains('Restaurant')]
Ristoranti.reset_index().drop(['index'], axis=1)

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,MATTARELLO - Pizzeria Forno a Legna,41.849479,12.458306,Italian Restaurant
1,La tana della volpe,41.844868,12.425431,Italian Restaurant
2,Il Bistrot,41.824133,12.450549,Italian Restaurant
3,Ilios,41.854703,12.478428,Greek Restaurant
4,Le Tre Zucche,41.862914,12.457062,Italian Restaurant
5,Aglio Olio E Peperoncino,41.831789,12.420041,Italian Restaurant
6,Mò Mò Republic,41.867142,12.450240,Italian Restaurant
7,Wild Ginger,41.827250,12.446370,Restaurant
8,L'Osteria di Monteverde,41.872937,12.455286,Mediterranean Restaurant
9,Retro 02,41.870020,12.443200,Italian Restaurant


In [17]:
Pizzerie=nearby_venues[nearby_venues['Venue Category']=='Pizza Place']
Pizzerie.reset_index().drop(['index'], axis=1)

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Smile Pizza,41.821029,12.443833,Pizza Place
1,Pane e Tempesta,41.868815,12.446032,Pizza Place
2,In Fucina,41.860618,12.452295,Pizza Place
3,Pizzeria i Gemelli,41.853582,12.475430,Pizza Place
4,Pappammí,41.865372,12.460642,Pizza Place
5,La Gatta Mangiona,41.875828,12.447631,Pizza Place
6,Le Quattro Stagioni,41.863543,12.466718,Pizza Place
7,Il Ponticello,41.851742,12.476499,Pizza Place
8,Alice Pizza Al Taglio,41.855681,12.479655,Pizza Place


In [19]:
# plots Restaurant (green circles) and Pizzerie (red circles) position and the center of the cluster
address = 'Roma'

#geolocator = Nominatim(user_agent="to_explorer")
#location = geolocator.geocode(address)
latitude = bari_lat
longitude = bari_lng

# create map of Roma using latitude and longitude values
map_Roma = folium.Map(location=[latitude, longitude], zoom_start=12)

label='Roma'
#maxdensita=Municipi['Densita'].max()

#for lat, lng, name, in zip(indirizzi['lat'], 
#                                   indirizzi['long'], 
#                                   indirizzi['Nome']):
#    label = '{}'.format(name)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=6,
#        popup=label,
#        color='green',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(map_Roma)  

for lat, long in center:
    label = 'Baricentro Numerelli {}'.format(center)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, long],
        #icon=folium.Icon(color=colors[center]),
        #icon=folium.Icon(color='red'),
        popup=label).add_to(map_Roma)
    
 
for lat, lng, name, in zip(Ristoranti['Venue Latitude'], 
                                   Ristoranti['Venue Longitude'], 
                                   Ristoranti['Venue']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Roma)  

for lat, lng, name, in zip(Pizzerie['Venue Latitude'], 
                                   Ristoranti['Venue Longitude'], 
                                   Ristoranti['Venue']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Roma) 

map_Roma